# Determines bounding boxes for each sulcus based on a mask

This notebook determines bounding box around a sulcus. The mask is built in order to only keep sulcus of interest. <br>
It uses a supervised database, in which each sulcus has been manually labelled.

# Imports

In [ ]:
import sys
import os
import json
import tempfile
import colorado as cld

In [ ]:
import anatomist.notebook as ana
a = ana.Anatomist()
print(a.headless_info.__dict__)

The following line permits to import deep_folding even if this notebook is executed from the notebooks subfolder (and no install has been launched):

 /notebooks/use_transform.ipynb  
 /deep_folding/__init__.py

In [ ]:
sys.path.append((os.path.abspath('../')))
import deep_folding
print((os.path.dirname(deep_folding.__file__)))

# User-specific variables

In [ ]:
sulcus = 'S.T.s.ter.asc.ant.'

In [ ]:
side = 'L'

We now assign path names and other user-specific variables.

The source directory is where the database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [ ]:
src_dir = os.path.join(os.getcwd(), '../data/source/supervised')
src_dir = os.path.abspath(src_dir)
print(("src_dir = " + src_dir))

In [ ]:
bbox_dir = os.path.join(os.getcwd(), '../data/target/bbox')
bbox_dir = os.path.abspath(bbox_dir)
print(("bbox_dir = " + bbox_dir))

In [ ]:
mask_dir = os.path.join(os.getcwd(), '../data/target/mask')
mask_dir = os.path.abspath(mask_dir)
print(("mask_dir = " + mask_dir))

In [ ]:
ref_dir = os.path.join(os.getcwd(), '../data/reference/bbox')
ref_dir = os.path.abspath(ref_dir)
print(("ref_dir = " + ref_dir))

In [ ]:
print((sys.argv))

Gets the normlized SPM file to get voxel size inside the program

norm_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
norm_dir = os.path.abspath(norm_dir)
sub_dir = "ANALYSIS/3T_morphologist/100206/t1mri/default_acquisition"

# Illustration of main program uses

### Using external calls

In [ ]:
!python ../deep_folding/anatomist_tools/mask.py --help

### By using the main function call

In [ ]:
from deep_folding.anatomist_tools import mask
print((mask.__file__))

In [ ]:
args = "--help"
argv = args.split(' ')

In [ ]:
mask.main(argv)

### By using the API function call

In [ ]:
mask.bounding_box(src_dir=src_dir,
                  bbox_dir=bbox_dir,
                  sulcus=sulcus,
                  side=side,
                  number_subjects=0)

# Test example

In [ ]:
unsupervised_dir = os.path.join(os.getcwd(), '../data/source/unsupervised')
reference_dir = os.path.join(os.getcwd(), '../data/reference')

In [ ]:
_, _, vol_mask = mask.bounding_box(src_dir=src_dir,
                                    bbox_dir=bbox_dir,
                                    mask_dir=mask_dir,
                                    sulcus=sulcus,
                                    side=side,
                                    number_subjects=1,
                                    out_voxel_size=1)

In [ ]:
vol_mask.shape 

In [ ]:
a_vol_mask = a.toAObject(vol_mask)
axial0 = a.createWindow("Axial")
axial0.addObjects(a_vol_mask)

In [ ]:
from soma import aims
import moving_averages as ma

temp_dir = tempfile.mkdtemp()
mask_filename_temp = f"{temp_dir}/mask.nii.gz"
aims.write(vol_mask, mask_filename_temp)
bucket_filename = f"{temp_dir}/mask.bck"
cmd = f"AimsFileConvert -c Bucket -t VOID -e 1 -i {mask_filename_temp} -o {bucket_filename}"
os.system(cmd)

# Displays bucket file
bucket, bucket_raw, dxyz, rot, tr = ma.load_bucket(bucket_filename)
m = cld.bucket_to_mesh(bucket)
cld.draw(m)


In [ ]:
cld.draw_numpy_bucket(bucket_raw)

# Mask test with more than 1 subject

In [ ]:
print(mask_dir)

In [ ]:
src_dir = "/host/neurospin/dico/data/bv_databases/human/pclean/all"

_, _, vol_mask = mask.bounding_box(src_dir=src_dir,
                                   bbox_dir=bbox_dir,
                                   mask_dir=mask_dir,
                                   sulcus=sulcus,
                                   side=side,
                                   number_subjects=10,
                                   out_voxel_size=2)

In [ ]:
c = aims.Converter_rc_ptr_Volume_S16_BucketMap_VOID()
bucket = c(vol_mask)

In [ ]:
# Displays bucket file
m = cld.bucket_to_mesh(bucket[0])
cld.draw(m)

We now represent the mask together with the MNI template:

In [ ]:
# We recover the MNI template
install_dir = "."
extracted_dir = f"{install_dir}/mni_icbm152_nlin_asym_09c"
if os.path.exists(extracted_dir):
    print(f'the directory {extracted_dir} already exists. Assuming it is OK.')
else:
    dl_url = "http://www.bic.mni.mcgill.ca/~vfonov/icbm/2009/mni_icbm152_nlin_asym_09c_nifti.zip"
    tmp_dl = tempfile.mkstemp(suffix='.zip')
    with urlopen(dl_url) as f:
        with open(tmp_dl[1], 'wb') as g:
            g.write(f.read())
    # Extract the archive
    with zipfile.ZipFile(tmp_dl[1], 'r') as zf:
        zf.extractall(install_dir)

In [ ]:
mni_file = f"{extracted_dir}/mni_icbm152_t1_tal_nlin_asym_09c.nii"
mni = a.loadObject(mni_file)

In [ ]:
mask_vol_aims = vol_mask
print(mask_vol_aims.header())

In [ ]:
# fusion 2D
mask_vol = a.toAObject(mask_vol_aims)
fusion2d = a.fusionObjects([mni, mask_vol], "Fusion2DMethod")
axial = a.createWindow("Axial")
axial.addObjects(fusion2d)
# params of the fusion : linear on non null
a.execute("Fusion2DParams", object=fusion2d, mode="linear_on_defined", rate=0.4)

# Result analysis

Prints the list of files of the target directory

In [ ]:
import os
bbox_dir_side = os.path.join(bbox_dir, side)
print(bbox_dir_side)
print(('\n'.join(os.listdir(bbox_dir_side))))

Expected output (we read the bounding_box file from the reference directory):

In [ ]:
ref_dir_side = os.path.join(ref_dir, side)
ref_file = os.listdir(ref_dir_side)[0]
print("ref bbox_file = ", ref_file, '\n')
with open(os.path.join(ref_dir_side,ref_file), 'r') as f:
    data_ref = json.load(f)
    print((json.dumps(data_ref, sort_keys=True, indent=4)))
    box_ref = {k: data_ref[k] for k in ['bbmin_voxel', 'bbmax_voxel']}

Obtained output (we read the bounding_box file from the target directory):

In [ ]:
bbox_file = os.listdir(bbox_dir_side)[0]
print("computed bbox_file = ", bbox_file, '\n')
with open(os.path.join(bbox_dir_side,bbox_file), 'r') as f:
    data_target = json.load(f)
    print((json.dumps(data_target, sort_keys=True, indent=4)))
    box_target = {k: data_target[k] for k in ('bbmin_voxel', 'bbmax_voxel')}

In [ ]:
box_target == box_ref

Somehow, it may be normal that the two boxes differ. They were coputed in different referentials